# <div align="center"> 平台 API 测试</div>

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from k12libs.utils.nb_easy import k12ai_post_request
from k12libs.utils.nb_easy import k12ai_get_data
from k12libs.utils.nb_easy import k12ai_print
from k12libs.utils.nb_easy import k12ai_train_execute
import time

## 1. 平台状态`/k12ai/platform/stats`


A. 输入

```json
{
    "op":"", // [M] api具有子集操作, 如: "query"
    "user": "", // [M] 用户ID, api调用者, 如: 用户账号user_id(不含有特殊字符)
    "service_name": "", // [M] 服务名, k12ai由多个子框架组成, 需要指定具体服务, 如: "k12nlp", "k12cv"
    "service_uuid": "", // [M] 服务唯一标识码, 可以是用户项目的工程ID, project_id.
    "async": true,  // [O] 默认同步, 即值为false, api调用过程方式
    "service_params": { // [M] 服务参数
        "cpus": true, // [O] k12ai主机cpu信息 
        "gpus": true, // [O] k12ai主机gpu信息
        "disks": false, // [O] k12ai主机磁盘信息
        "containers": true // [O] k12ai主机上正在运行的label为"k12ai.service.name"的container信息
        "services": false, // [O] 用户user当前的运行的服务信息, 当service_uuid='*',返回所有user的services
    }
}
```

B. 异步输出

```json
{
    "version": "0.1.0", // [M] 返回json格式的版本号, 兼容版本时使用
    "type": "result",   // [M] 消息体类型
    "tag": "platform",  // [M] 标识输出结果的服务TAG, 可以用来后续消息过滤 
    "op": "stats",      // [M] 标识输出结果由哪个功能产出, 一般统一为"/k12ai/{{tag}}/{{op}}"的形式
    "user": "",         // [M] 用户ID
    "service_uuid": "", // [M] 服务唯一标识码
    "timestamp": , // [M]
    "datetime": , // [M]
    "data": { // [M] 输出结果的正文内容
        "code": 100000,
        "descr": "task service success",
        "message": {
            "ip": "10.255.0.58",  // [M] k12ai主机的ip
            "cpu_percent": 47.4,  // [C] k12ai主机cpu使用率, 不区分多cpu 
            "cpu_percent_list": [ // [C] k12ai主机cpu使用率, 每个cpu的使用率
                63.6,
                31.1
            ],
            "cpu_memory_total":, // [C] k12ai主机memory总大小(Byte)
            "cpu_memory_usage":, // [C] k12ai主机memory已使用(Byte)
            "cpu_memory_percent": , // [C] k12ai主机memory使用率
            "gpus": [  // [C] k12ai主机gpu信息, 每个gpu信息列表
                {
                    "name": "", // [C] gpu名字, 厂商
                    "gpu_percent": , // [C] gpu使用率
                    "gpu_memory_total": , // [C] gpu显存总大小(Byte)
                    "gpu_memory_usage": , // [C] gpu显存已使用(Byte)
                    "gpu_memory_percent": // [C] gpu显存使用率
                }
            ],
            "containers": [ // k12ai主机正在运行的关于训练相关的container信息
                {
                    "id": "", // container的ID
                    "op": "",  // container正在执行的操作,如:"train.start", "predict.start"等
                    "user": "", // container用户ID
                    "service_uuid": "" // container执行的任务/服务ID.
                    "cpu_percent": , // container的cpu使用率
                    "cpu_memory_total":,  // container的memory总大小
                    "cpu_memory_usage":,  // container的memory已使用
                    "cpu_memory_percent":, // container的memory利用率
                    "gpu_percent": , // container的gpu使用率
                    "gpu_memory_usage":,  // container的gpu memory已使用
                    "gpu_memory_percent":, // container的gpu memory利用率
                }
            ]
           "services": [ // user的正在运行的服务信息
                {
                    "id": "0c6475f3872d",  // 运行服务的container的ID
                    "op": "train.start",   // container正在执行的操作
                    "service_uuid": "mnist", // 服务ID.
                    "service_pid": 19773, //  服务PID.
                    "service_starttime": "2020-01-03T13:20:11.830676445Z"  // 服务启动时间
                }
            ]
        }
    }
}
```

C. 同步输出:

```json
{
    "code": 100000,
    "descr": "task service success",
    "message": {
        "ip": "10.255.0.58",  // [M] k12ai主机的ip
        "cpu_percent": 47.4,  // [C] k12ai主机cpu使用率, 不区分多cpu 
        "cpu_percent_list": [ // [C] k12ai主机cpu使用率, 每个cpu的使用率
            63.6,
            31.1
        ],
        "cpu_memory_total":, // [C] k12ai主机memory总大小(Byte)
        "cpu_memory_usage":, // [C] k12ai主机memory已使用(Byte)
        "cpu_memory_percent": , // [C] k12ai主机memory使用率
        "gpus": [  // [C] k12ai主机gpu信息, 每个gpu信息列表
            {
                "name": "", // [C] gpu名字, 厂商
                "gpu_percent": , // [C] gpu使用率
                "gpu_memory_total": , // [C] gpu显存总大小(Byte)
                "gpu_memory_usage": , // [C] gpu显存已使用(Byte)
                "gpu_memory_percent": // [C] gpu显存使用率
            }
        ],
        "containers": [ // k12ai主机正在运行的关于训练相关的container信息
            {
                "id": "", // container的ID
                "op": "",  // container正在执行的操作,如:"train.start", "predict.start"等
                "user": "", // container用户ID
                "service_uuid": "" // container执行的任务/服务ID.
                "cpu_percent": , // container的cpu使用率
                "cpu_memory_total":,  // container的memory总大小
                "cpu_memory_usage":,  // container的memory已使用
                "cpu_memory_percent":, // container的memory利用率
                "gpu_percent": , // container的gpu使用率
                "gpu_memory_usage":,  // container的gpu memory已使用
                "gpu_memory_percent":, // container的gpu memory利用率
            }
        ]
    }
}
```

In [3]:
user = '15801310416'
uuid = '0'
query_key = f'framework/{user}/{uuid}/query/result'
train_keys = k12ai_train_execute(iter_num=10, run_num=2)
time.sleep(5)

In [4]:
isasync = True
query = '''{
    "cpus": true,
    "gpus": true,
    "disks": false,
    "containers": true
}'''

stats = k12ai_post_request(
    uri='k12ai/platform/stats',
    data = {
        'op': 'query',
        'user': user,
        'service_uuid': uuid,
        'service_params': query,
        'isasync': isasync
    }
)
k12ai_print(stats)
if isasync:
    print("Async Out:")
    k12ai_print(k12ai_get_data(query_key, waitcnt=5))

{
    "code": 100000,
    "message": {
        "en": "success",
        "cn": "成功"
    }
}
Async Out:
[
    {
        "key": "framework/15801310416/0/query/result/20200310123323",
        "value": {
            "version": "0.1.0",
            "type": "result",
            "token": "0",
            "user": "15801310416",
            "op": "query",
            "service_name": "k12ai",
            "service_uuid": "0",
            "timestamp": 1583814803149,
            "datetime": "20200310123323",
            "data": {
                "code": 100000,
                "message": {
                    "en": "success",
                    "cn": "成功"
                },
                "data": {
                    "ip": "10.255.0.58",
                    "cpu_percent": 50.5,
                    "cpu_percent_list": [
                        51.6,
                        50.7,
                        50.0,
                        50.7,
                        51.4,
                        49.3,

In [5]:
isasync = False
query = '''{
    "cpus": true,
    "gpus": true,
    "disks": false,
    "containers": true
}'''

stats = k12ai_post_request(
    uri='k12ai/platform/stats',
        data = {
        'op': 'query',
        'user': user,
        'service_uuid': uuid,
        'service_params': query,
        'isasync': isasync
    }
)
k12ai_print(stats)

{
    "code": 100000,
    "message": {
        "en": "success",
        "cn": "成功"
    },
    "data": {
        "ip": "10.255.0.58",
        "cpu_percent": 28.3,
        "cpu_percent_list": [
            4.5,
            98.8,
            4.0,
            98.8,
            4.0,
            5.4,
            4.5,
            5.5
        ],
        "cpu_memory_total": 50648920064,
        "cpu_memory_usage": 9042296832,
        "cpu_memory_percent": 19.8,
        "gpus": [
            {
                "name": "Tesla P40",
                "gpu_percent": 8.0,
                "gpu_memory_total": 22919.0,
                "gpu_memory_usage": 714.0,
                "gpu_memory_percent": 3.12
            }
        ],
        "containers": [
            {
                "id": "b4681a826453",
                "op": "train.start",
                "user": "15801310416",
                "service_uuid": "598b39-1",
                "cpu_percent": 101.04,
                "cpu_memory_total": 10737418240

### 1.2 获取用户的service信息

In [6]:
isasync = False
query = '''{
    "services": true
}'''

stats = k12ai_post_request(
    uri='k12ai/platform/stats',
    data = {
        'op': 'query',
        'user': user,
        'service_uuid': uuid,
        'service_params': query,
        'isasync': isasync
    }
)
k12ai_print(stats)

{
    "code": 100000,
    "message": {
        "en": "success",
        "cn": "成功"
    },
    "data": {
        "services": [
            {
                "id": "b4681a826453",
                "op": "train.start",
                "service_uuid": "598b39-1",
                "service_pid": 23645,
                "service_starttime": "2020-03-10T04:33:15.378161962Z"
            },
            {
                "id": "0446caef7713",
                "op": "train.start",
                "service_uuid": "598b39-0",
                "service_pid": 23588,
                "service_starttime": "2020-03-10T04:33:15.316566208Z"
            }
        ]
    }
}


## 2. 平台控制/k12ai/platform/control

A. 输入

```json
{
    "op": "",  // [M] 控制命令, 如, 停止某个container任务"container.stop"
    "user": "", // [M] 用户ID, api调用者, 如: 用户账号user_id(不含有特殊字符)
    "service_name": "", // [M] 服务名, k12ai由多个子框架组成, 需要指定具体服务, 如: "k12nlp", "k12cv"
    "service_uuid": "", // [M] 服务唯一标识码, 可以是用户项目的工程ID, project_id.
    "async": true, // [O] 控制命令的是否异步操作, 提醒: 有些命令操作是耗时的, 需要采用异步方式
    "service_params": { // [M] 服务参数
    }
}
```

B. 输出

```json
{
    "code": 100000,
    "memsage": {},
    "data": ""
}
```

### 2.1 停止container

In [7]:
result = k12ai_get_data(query_key, rm=True)
try:
    cid = result[0]['value']['data']['data']['services'][0]['id']
except Exception:
    cid = 'unknow'

isasync = False
stats = k12ai_post_request(
    uri='k12ai/platform/control',
    data = {
        'op': 'container.stop',
        'user': user,
        'service_uuid': uuid,
        'service_params': '{"id":"%s"}' % cid,
        'isasync': isasync
    }
)
k12ai_print(stats)

[{'key': 'framework/15801310416/0/query/result/20200310123331', 'value': {'version': '0.1.0', 'type': 'result', 'token': '0', 'user': '15801310416', 'op': 'query', 'service_name': 'k12ai', 'service_uuid': '0', 'timestamp': 1583814811274, 'datetime': '20200310123331', 'data': {'code': 100000, 'message': {'en': 'success', 'cn': '成功'}, 'data': {'services': [{'id': 'b4681a826453', 'op': 'train.start', 'service_uuid': '598b39-1', 'service_pid': 23645, 'service_starttime': '2020-03-10T04:33:15.378161962Z'}, {'id': '0446caef7713', 'op': 'train.start', 'service_uuid': '598b39-0', 'service_pid': 23588, 'service_starttime': '2020-03-10T04:33:15.316566208Z'}]}}}}]
{
    "code": 100000,
    "message": {
        "en": "success",
        "cn": "成功"
    }
}
